<a href="https://colab.research.google.com/github/Mr-Kondo/_Inbox/blob/main/local_llm_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
from huggingface_hub import login

token = userdata.get("HUGGINGFACE_TOKEN")
login(token=token)
print("HF login OK")

HF login OK


In [ ]:
def build_agent():
    from langchain_core.prompts import PromptTemplate

    # シンプルなReActテンプレート
    template = """You have access to these tools:
{tools}

Use this EXACT format:
Question: [question]
Thought: [your reasoning]
Action: calculator
Action Input: [expression]
Observation: [result]
Final Answer: [number only]

Question: {input}
{agent_scratchpad}"""

    prompt = PromptTemplate.from_template(template)
    agent = create_react_agent(CHAT, [calculator], prompt)

    exec_ = AgentExecutor(
        agent=agent, tools=[calculator],
        return_intermediate_steps=True,
        handle_parsing_errors=True,
        max_iterations=5,  # 30→5に削減（無限ループ防止）
        max_execution_time=20,  # 45→20秒に短縮
        early_stopping_method="force",
        verbose=True  # デバッグ用
    )
    return exec_


In [ ]:
# モデルが基本的な指示に従えるかテスト
test_prompts = [
    "Output only the number 42:",
    "What is 2+2? Answer with only a number:",
    '{"ok": true}',
]

print("\n=== BASIC MODEL TEST ===")
for p in test_prompts:
    messages = [{"role": "user", "content": p}]
    formatted = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    result = gen_pipe(formatted, max_new_tokens=50, return_full_text=False)
    output = result[0]["generated_text"]
    print(f"Prompt: {p}")
    print(f"Output: {output}")
    print("-"*60)


=== BASIC MODEL TEST ===
Prompt: Output only the number 42:
Output: 42
------------------------------------------------------------
Prompt: What is 2+2? Answer with only a number:
Output: 4
------------------------------------------------------------
Prompt: {"ok": true}
Output: It seems like you're providing a JSON object with a single key-value pair. However, it doesn't appear to be a complete or meaningful object. Can you please provide more context or clarify what you're trying to accomplish? I'll do my best
------------------------------------------------------------


In [4]:
# ==== (0) 依存（必要に応じて実行） =========================================
!pip -q install -U transformers accelerate bitsandbytes sentencepiece huggingface_hub
!pip -q install -U langchain langchain-huggingface langgraph


# ==== (1) 共通セットアップ ====================================================
import os
import warnings
import traceback
import sys

# LangSmith完全無効化
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = ""
warnings.filterwarnings('ignore', category=UserWarning, module='langsmith')

import csv, json, time, re, datetime, pathlib
import torch
from time import perf_counter
from typing import Dict, Any, List

# ---- HFトークン取得 ---------------------------------------------------------
HF_TOKEN = None
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
    print("✓ HF token loaded from Colab Secrets")
except Exception:
    try:
        from kaggle_secrets import UserSecretsClient
        HF_TOKEN = UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")
        print("✓ HF token loaded from Kaggle Secrets")
    except Exception:
        HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
        if HF_TOKEN:
            print("✓ HF token loaded from environment")

# ---- HF 4bitロード（NF4） ---------------------------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, token=HF_TOKEN)
mdl = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_cfg,
    torch_dtype=torch.float16,  # ✅ 修正：torch_dtype → dtype警告を回避
    token=HF_TOKEN
)

# Llama系はpad未定義→pad=eosを明示
tok.pad_token_id = tok.eos_token_id
mdl.generation_config.pad_token_id = tok.pad_token_id

# ✅ 修正：GEN_KWをパイプライン作成「前」に定義
GEN_KW = dict(
    max_new_tokens=256,  # ✅ 128→256
    do_sample=False,
    pad_token_id=tok.pad_token_id,
    return_full_text=False,
)

# 警告抑制
import logging
logging.getLogger("transformers.pipelines").setLevel(logging.ERROR)

# 生成パイプライン
gen_pipe = pipeline("text-generation", model=mdl, tokenizer=tok, **GEN_KW)

# メモリ確認
import gc
torch.cuda.empty_cache()
gc.collect()

print(f"GPU allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
print(f"GPU reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

# ---- LangChainラッパ ---------------------------------------------------------
from langchain_huggingface import HuggingFacePipeline

# ChatHuggingFaceを使わず、HuggingFacePipelineを直接使用
LLM = HuggingFacePipeline(pipeline=gen_pipe)

# チャット形式を手動で適用するヘルパー関数
def format_for_llama(prompt: str) -> str:
    """Llama-3.2のチャット形式に変換"""
    messages = [{"role": "user", "content": prompt}]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("✓ Model and pipeline ready\n")


# ==== (2) ツール／Runner実装 ==================================================

# ✅ safe_calc関数を最初に定義
def safe_calc(expr: str) -> str:
    """Evaluate arithmetic expression safely."""
    expr = expr.replace("^", "**")  # べき乗演算子を変換
    # 安全な文字のみ許可
    if not re.fullmatch(r"[0-9\+\-\*/\(\)\s\*]+", expr):
        return "ERROR: invalid characters"
    try:
        # 安全な環境でeval実行
        val = eval(expr, {"__builtins__": {}}, {})
        return str(int(val))
    except Exception as e:
        return f"ERROR: {e}"

# --- Chain (LCEL) -------------------------------------------------------------
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def run_chain(prompt: str) -> Dict[str, Any]:
    try:
        print(f"[CHAIN] Starting...", flush=True)

        # 直接フォーマットして実行
        formatted = format_for_llama(prompt)

        t0 = perf_counter()
        result = gen_pipe(formatted)
        out = result[0]["generated_text"].strip()
        t1 = perf_counter()

        print(f"[CHAIN] Output: {out[:80]}", flush=True)
        return {"output": out, "ms": int((t1 - t0) * 1000), "tool_calls": 0}
    except Exception as e:
        print(f"[CHAIN ERROR] {e}", flush=True)
        traceback.print_exc()
        return {"output": f"ERROR: {e}", "ms": 0, "tool_calls": 0}

# --- Agent (ReAct) ------------------------------------------------------------
from langchain import hub
try:
    from langchain.tools import tool
except Exception:
    from langchain_core.tools import tool

@tool("calculator")
def calculator(expr: str) -> str:
    """Evaluate arithmetic like '231*47 + 5^3' and return the integer result."""
    return safe_calc(expr)

from langchain.agents import create_react_agent, AgentExecutor

@tool("calculator")
def calculator(expr: str) -> str:
    """Evaluate arithmetic like '231*47 + 5^3' and return the integer result."""
    return safe_calc(expr)

def run_agent(prompt: str) -> Dict[str, Any]:
    """簡易Agent：ツールが必要そうなら呼び出す、そうでなければ直接回答"""
    try:
        print(f"[AGENT] Starting...", flush=True)
        t0 = perf_counter()
        tool_calls = 0

        # ステップ1: LLMに判断させる
        decision_prompt = f"""Task: {prompt}

If this needs calculation, respond ONLY:
USE_CALC: expression

Otherwise respond ONLY:
ANSWER: your answer"""

        formatted = format_for_llama(decision_prompt)
        result = gen_pipe(formatted, max_new_tokens=64)
        response = result[0]["generated_text"].strip()

        print(f"[AGENT] LLM response: {response[:100]}", flush=True)

        # ステップ2: ツール呼び出しの判定
        if "USE_CALC:" in response:
            # 式を抽出
            expr = response.split("USE_CALC:", 1)[1].strip().split()[0]
            print(f"[AGENT] Calling calculator: {expr}", flush=True)
            calc_result = calculator.invoke(expr)
            tool_calls = 1

            # ステップ3: 結果を整形
            final_prompt = f"The calculation result is {calc_result}. Output only this number:"
            formatted2 = format_for_llama(final_prompt)
            result2 = gen_pipe(formatted2, max_new_tokens=32)
            final_answer = result2[0]["generated_text"].strip()
        else:
            # ANSWER:から抽出
            if "ANSWER:" in response:
                final_answer = response.split("ANSWER:", 1)[1].strip()
            else:
                final_answer = response

        t1 = perf_counter()
        print(f"[AGENT] Final: {final_answer[:80]}", flush=True)

        return {
            "output": final_answer,
            "ms": int((t1 - t0) * 1000),
            "tool_calls": tool_calls,
        }
    except Exception as e:
        print(f"[AGENT ERROR] {e}", flush=True)
        traceback.print_exc()
        return {"output": f"ERROR: {e}", "ms": 0, "tool_calls": 0}


# --- LangGraph（修正版） ---------------------------------------------------
from langgraph.graph import StateGraph, END
from langgraph.errors import GraphRecursionError
from langchain_core.messages import HumanMessage, AIMessage

GRAPH_INSTR = """You can use the 'calculator' tool.

If calculation needed, respond ONLY:
{"action":"calculator","action_input":"231*47+5^3"}

Otherwise respond:
Final Answer: [your answer]"""

def _ai_says_action(txt: str):
    try:
        start = txt.index("{")
        end = txt.rindex("}") + 1
        js = json.loads(txt[start:end])
        if isinstance(js, dict) and js.get("action") == "calculator":
            return js.get("action_input", "")
    except Exception:
        pass
    return None

def build_graph():
    try:
        print("[GRAPH] Building...", flush=True)
        g = StateGraph(dict)

        def llm_node(state: dict) -> dict:
            # メッセージからテキスト抽出してフォーマット
            last_msg = state["messages"][-1]
            if isinstance(last_msg, HumanMessage):
                prompt = last_msg.content
            else:
                prompt = str(last_msg)

            formatted = format_for_llama(prompt)
            result = gen_pipe(formatted)
            resp_text = result[0]["generated_text"].strip()

            state["messages"].append(AIMessage(content=resp_text))
            return state

        def route(state: dict):
            last = state["messages"][-1]
            if isinstance(last, AIMessage):
                txt = (last.content or "").strip()
                act = _ai_says_action(txt)
                if act is not None:
                    return "tools"
                if "Final Answer:" in txt:
                    return END
            return "llm"

        def tools_node(state: dict) -> dict:
            last = state["messages"][-1].content
            expr = _ai_says_action(last) or ""
            result = calculator.invoke(expr)
            obs = AIMessage(content=f"Observation: {result}\nFinal Answer: {result}")
            state["messages"].append(obs)
            state["tool_calls"] = state.get("tool_calls", 0) + 1
            return state

        g.add_node("llm", llm_node)
        g.add_node("tools", tools_node)
        g.add_conditional_edges("llm", route, {"llm": "llm", "tools": "tools", END: END})
        g.add_edge("tools", END)
        g.set_entry_point("llm")

        compiled = g.compile()
        print("[GRAPH] ✓ Graph ready", flush=True)
        return compiled
    except Exception as e:
        print(f"[GRAPH BUILD ERROR] {e}", flush=True)
        traceback.print_exc()
        raise

GRAPH_APP = build_graph()

def run_graph(prompt: str, rec_limit: int = 25) -> Dict[str, Any]:
    try:
        print(f"[GRAPH] Starting...", flush=True)
        t0 = perf_counter()
        state = {"messages": [HumanMessage(content=f"{GRAPH_INSTR}\n\nQuestion: {prompt}")]}
        out_state = GRAPH_APP.invoke(state, config={"recursion_limit": rec_limit})
        t1 = perf_counter()

        final = ""
        for m in reversed(out_state["messages"]):
            if isinstance(m, AIMessage):
                final = m.content or ""
                break

        tools_used = out_state.get("tool_calls", 0)
        print(f"[GRAPH] Completed, tools={tools_used}", flush=True)
        return {"output": final, "ms": int((t1 - t0) * 1000), "tool_calls": tools_used}

    except GraphRecursionError:
        print(f"[GRAPH] Recursion limit", flush=True)
        return {"output": "ERROR: recursion limit", "ms": int((perf_counter()-t0)*1000), "tool_calls": 0}
    except Exception as e:
        print(f"[GRAPH ERROR] {e}", flush=True)
        traceback.print_exc()
        return {"output": f"ERROR: {e}", "ms": 0, "tool_calls": 0}


## ==== (3) タスク定義＆評価 =====================================================

# ✅ 評価関数を先に定義
def eval_json_ok(s: str) -> bool:
    """JSONが {"ok": true} であるかチェック"""
    try:
        # 余計なテキストを除去してJSON部分を抽出
        s = s.strip()
        # {"ok": true} だけの場合
        if s == '{"ok": true}' or s == "{'ok': True}":
            return True
        # JSONパース試行
        obj = json.loads(s)
        return isinstance(obj, dict) and obj.get("ok") is True
    except Exception:
        return False

def eval_eq_int(s: str, expect: int) -> bool:
    """出力が期待する整数と一致するかチェック"""
    s = s.strip()

    # ✅ "Observation: 数字" パターンを追加
    if "Observation:" in s:
        # "Observation: 10982 Final Answer: 10982" から数字を抽出
        obs_match = re.search(r'Observation:\s*(\d+)', s)
        if obs_match:
            try:
                return int(obs_match.group(1)) == expect
            except:
                pass

    # 既存のロジック
    if s.startswith("Final Answer:"):
        s = s.split("Final Answer:", 1)[1].strip()

    try:
        num_str = re.sub(r"[^\d\-]", "", s)
        if not num_str:
            return False
        return int(num_str) == expect
    except Exception:
        return False
        return False

def eval_contains_one(s: str, options: List[str]) -> bool:
    """出力がoptions内の1つの単語と完全一致するかチェック"""
    s = s.strip()
    if s.startswith("Final Answer:"):
        s = s.split(":", 1)[1].strip()
    # 余計な句読点を除去
    s = re.sub(r'[。、\s]+', '', s)
    return s in options

def eval_email(s: str, expect: str) -> bool:
    """メールアドレスを抽出してチェック"""
    s = s.strip()
    if s.startswith("Final Answer:"):
        s = s.split(":", 1)[1].strip()
    # メールアドレスパターンを抽出
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    matches = re.findall(email_pattern, s)
    if matches:
        return matches[0].lower() == expect.lower()
    # 直接一致もチェック
    return s.lower() == expect.lower()

# ✅ タスク定義（評価関数の後に配置）
ALL_TASKS = {
    "s0_format": {
        "prompt": 'Output EXACTLY: {"ok": true}',
        "eval": lambda s: eval_json_ok(s),
    },
    "s1_reasoning": {
        "prompt": "What is 40+2? Output only the number:",
        "eval": lambda s: eval_eq_int(s, 42),
    },
    "s3_tool": {
        "prompt": "Calculate: 231*47 + 5^3. Output only the final integer:",
        "eval": lambda s: eval_eq_int(s, 10982),
    },
    "s4_context_qa": {
        "prompt": "Context: 日本の古都として有名なのは京都と奈良です。Extract ONE city name. Output only one word:",
        "eval": lambda s: eval_contains_one(s, ["京都", "奈良"]),
    },
    "s5_extract_email": {
        "prompt": "Text: 連絡先は info@example.com です。Extract the email address:",
        "eval": lambda s: eval_email(s, "info@example.com"),
    },
}


#==== (4) 実行ループ（すべてのタスク×3方式） ================================
RUN_TASKS = list(ALL_TASKS.keys())  # 実行したいタスクを絞る場合は編集
RUNNERS = {
    "chain": run_chain,
    "agent": run_agent,
    "graph": run_graph,
}

stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_dir = pathlib.Path("results_"+stamp); out_dir.mkdir(exist_ok=True)
csv_path = out_dir / "results.csv"

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "trial","runner","task","wall_time_ms","tool_calls","passed","output"
    ])
    writer.writeheader()
    TRIALS = 3  # 回数は適宜変更
    for t in range(1, TRIALS+1):
        for task_name in RUN_TASKS:
            P = ALL_TASKS[task_name]["prompt"]
            E = ALL_TASKS[task_name]["eval"]
            for rname, fn in RUNNERS.items():
                res = fn(P)
                passed = bool(E(res["output"]))
                writer.writerow({
                    "trial": t, "runner": rname, "task": task_name,
                    "wall_time_ms": res["ms"], "tool_calls": res["tool_calls"],
                    "passed": int(passed), "output": res["output"][:200].replace("\n"," "),
                })
                print(f"[t{t}] {rname:<5} | {task_name:<14} | {res['ms']:>5} ms | tools={res['tool_calls']} | pass={passed}")

print(f"\nSaved -> {csv_path}")

✓ HF token loaded from Colab Secrets


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPU allocated: 4.75 GB
GPU reserved: 6.08 GB
✓ Model and pipeline ready

[GRAPH] Building...
[GRAPH] ✓ Graph ready
[CHAIN] Starting...
[CHAIN] Output: {"ok": true}
[t1] chain | s0_format      |  1671 ms | tools=0 | pass=True
[AGENT] Starting...
[AGENT] LLM response: ANSWER: {"ok": true}
[AGENT] Final: {"ok": true}
[t1] agent | s0_format      |   742 ms | tools=0 | pass=True
[GRAPH] Starting...
[GRAPH] Recursion limit
[t1] graph | s0_format      | 355519 ms | tools=0 | pass=False
[CHAIN] Starting...
[CHAIN] Output: 42
[t1] chain | s1_reasoning   |   147 ms | tools=0 | pass=True
[AGENT] Starting...
[AGENT] LLM response: ANSWER: 42
[AGENT] Final: 42
[t1] agent | s1_reasoning   |   382 ms | tools=0 | pass=True
[GRAPH] Starting...
[GRAPH] Completed, tools=0
[t1] graph | s1_reasoning   |   387 ms | tools=0 | pass=True
[CHAIN] Starting...
[CHAIN] Output: To calculate the expression, we need to follow the order of operations (PEMDAS):
[t1] chain | s3_tool        |  4488 ms | tools=0 | pass=Fal

In [5]:
import pandas as pd

# 最新のresults.csvを読み込み
csv_files = sorted(pathlib.Path(".").glob("results_*/results.csv"))
if csv_files:
    df = pd.read_csv(csv_files[-1])

    # 各Runnerの代表的な出力を表示
    print("="*80)
    print("SAMPLE OUTPUTS (First trial only)")
    print("="*80)

    for task in ["s0_format", "s1_reasoning", "s3_tool"]:
        print(f"\n### Task: {task} ###\n")
        subset = df[(df["trial"] == 1) & (df["task"] == task)]
        for _, row in subset.iterrows():
            print(f"[{row['runner']:>5}] {row['output'][:300]}")
            print("-"*80)
else:
    print("No results CSV found")

SAMPLE OUTPUTS (First trial only)

### Task: s0_format ###

[chain] {"ok": true}
--------------------------------------------------------------------------------
[agent] {"ok": true}
--------------------------------------------------------------------------------
[graph] ERROR: recursion limit
--------------------------------------------------------------------------------

### Task: s1_reasoning ###

[chain] 42
--------------------------------------------------------------------------------
[agent] 42
--------------------------------------------------------------------------------
[graph] Final Answer: 42
--------------------------------------------------------------------------------

### Task: s3_tool ###

[chain] To calculate the expression, we need to follow the order of operations (PEMDAS):  1. Calculate the exponentiation: 5^3 = 125 2. Multiply 231 by 47: 231 * 47 = 10827 3. Add 125 to 10827: 10827 + 125 = 
------------------------------------------------------------------------

In [6]:
# 最新のCSVを読み込んで分析
import pandas as pd

csv_files = sorted(pathlib.Path(".").glob("results_*/results.csv"))
if csv_files:
    df = pd.read_csv(csv_files[-1])

    print("="*80)
    print("SUCCESS RATE BY RUNNER")
    print("="*80)
    summary = df.groupby('runner').agg({
        'passed': ['sum', 'count', 'mean']
    }).round(3)
    print(summary)

    print("\n" + "="*80)
    print("SUCCESS RATE BY TASK")
    print("="*80)
    summary2 = df.groupby('task').agg({
        'passed': ['sum', 'count', 'mean']
    }).round(3)
    print(summary2)

    print("\n" + "="*80)
    print("FAILED TASKS - SAMPLE OUTPUTS")
    print("="*80)
    failed = df[(df['passed'] == 0) & (df['trial'] == 1)]
    for _, row in failed.head(10).iterrows():
        print(f"\n[{row['runner']:>5}] {row['task']}")
        print(f"Output: {row['output'][:150]}")

SUCCESS RATE BY RUNNER
       passed           
          sum count mean
runner                  
agent      12    15  0.8
chain      12    15  0.8
graph       9    15  0.6

SUCCESS RATE BY TASK
                 passed             
                    sum count   mean
task                                
s0_format             6     9  0.667
s1_reasoning          9     9  1.000
s3_tool               3     9  0.333
s4_context_qa         6     9  0.667
s5_extract_email      9     9  1.000

FAILED TASKS - SAMPLE OUTPUTS

[graph] s0_format
Output: ERROR: recursion limit

[chain] s3_tool
Output: To calculate the expression, we need to follow the order of operations (PEMDAS):  1. Calculate the exponentiation: 5^3 = 125 2. Multiply 231 by 47: 23

[agent] s3_tool
Output: 10947

[graph] s4_context_qa
Output: ERROR: recursion limit
